In [ ]:
%pip install transformers
%pip install tqdm
%pip install contractions

In [19]:
import pandas as pd
import numpy as np
import re
import nltk
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
import torch
from transformers import BertTokenizer
from transformers import BertModel
from tqdm import tqdm 

In [5]:
def load_data(filepath):
    df = pd.read_csv(filepath)
    return df

In [6]:
small_df = load_data('Data/CDIAL-Bias-English/train/small_df.csv')
complete_df = load_data('Data/CDIAL-Bias-English/train/complete_df.csv')

In [11]:
complete_df = complete_df.sort_values(by=['idx']).sample(frac=1).reset_index(drop=True)
small_df = small_df.sort_values(by=['idx']).sample(frac=1).reset_index(drop=True)

In [12]:
complete_df

,topic,idx,q,a,context,datatype,attitude,group
0,occupation,26876,What if a nursing student doesn't want to be a...,I also have the same idea as you. I am a fresh...,0,0,0,Nursing careers
1,region,210,Are Shanghainese really richer?,"A very serious answer, this question should be...",0,1,2,Shanghainese
2,Gender,7902,How do lesbians make life after 30 not pathetic?,"Invincible and comfortable, I thought about ma...",1,2,2,Homosexual/Lesbian
3,Gender,7658,Where is the way out for homosexuals in China?,I was lucky. The girl has come out with her pa...,1,1,2,Homosexual
4,Gender,8739,Is there any way to discern LES?,"I don't know, I just know that I like my siste...",1,0,0,Lesbian
...,...,...,...,...,...,...,...,...
19719,Ethnic culture,20848,How do you view foreigners thinking of Chinese...,Why didn't I understand that? Shiro is not a r...,0,0,0,Complexion
19720,region,1795,What do Northeasterners think of the Northeast...,Hate that iron is not steel. As long as there ...,1,1,3,Northeasterners
19721,Gender,13725,What do you think of the unabashed discriminat...,"There is no need to ask this question, feminis...",0,2,2,Discrimination against men
19722,Gender,10507,Want to know what it's like to have a stay-at-...,"I asked my husband, and he said that I was abl...",0,1,2,housewife


In [54]:
complete_df['concat'] = complete_df['q'] + complete_df['a']
complete_df

,topic,idx,q,a,context,datatype,attitude,group,output,tokenList,concat
0,occupation,26876,What if a nursing student doesn't want to be a...,I also have the same idea as you. I am a fresh...,0,0,0,Nursing careers,0,"[input_ids, token_type_ids, attention_mask]",What if a nursing student doesn't want to be a...
1,region,210,Are Shanghainese really richer?,"A very serious answer, this question should be...",0,1,2,Shanghainese,6,"[input_ids, token_type_ids, attention_mask]",Are Shanghainese really richer?A very serious ...
2,Gender,7902,How do lesbians make life after 30 not pathetic?,"Invincible and comfortable, I thought about ma...",1,2,2,Homosexual/Lesbian,22,"[input_ids, token_type_ids, attention_mask]",How do lesbians make life after 30 not patheti...
3,Gender,7658,Where is the way out for homosexuals in China?,I was lucky. The girl has come out with her pa...,1,1,2,Homosexual,18,"[input_ids, token_type_ids, attention_mask]",Where is the way out for homosexuals in China?...
4,Gender,8739,Is there any way to discern LES?,"I don't know, I just know that I like my siste...",1,0,0,Lesbian,12,"[input_ids, token_type_ids, attention_mask]","Is there any way to discern LES?I don't know, ..."
...,...,...,...,...,...,...,...,...,...,...,...
19719,Ethnic culture,20848,How do you view foreigners thinking of Chinese...,Why didn't I understand that? Shiro is not a r...,0,0,0,Complexion,0,"[input_ids, token_type_ids, attention_mask]",How do you view foreigners thinking of Chinese...
19720,region,1795,What do Northeasterners think of the Northeast...,Hate that iron is not steel. As long as there ...,1,1,3,Northeasterners,19,"[input_ids, token_type_ids, attention_mask]",What do Northeasterners think of the Northeast...
19721,Gender,13725,What do you think of the unabashed discriminat...,"There is no need to ask this question, feminis...",0,2,2,Discrimination against men,10,"[input_ids, token_type_ids, attention_mask]",What do you think of the unabashed discriminat...
19722,Gender,10507,Want to know what it's like to have a stay-at-...,"I asked my husband, and he said that I was abl...",0,1,2,housewife,6,"[input_ids, token_type_ids, attention_mask]",Want to know what it's like to have a stay-at-...


In [55]:
X = complete_df['concat']
X = X.astype(str)
X

0        What if a nursing student doesn't want to be a...
1        Are Shanghainese really richer?A very serious ...
2        How do lesbians make life after 30 not patheti...
3        Where is the way out for homosexuals in China?...
4        Is there any way to discern LES?I don't know, ...
                               ...                        
19719    How do you view foreigners thinking of Chinese...
19720    What do Northeasterners think of the Northeast...
19721    What do you think of the unabashed discriminat...
19722    Want to know what it's like to have a stay-at-...
19723    When did you feel discrimination against men?A...
Name: concat, Length: 19722, dtype: object

In [13]:

def get_encoded_labels(label):
    dict_map = {'000': 0, '001':1, '002':2, '003':3, '010':4, '011':5, '012':6, '013':7, '020':8, '021':9, '022':10,'023':11,
            '100': 12, '101':13, '102':14, '103':15, '110':16, '111':17, '112':18, '113':19, '120':20, '121':21, '122':22,'123':23}
    return dict_map[label]

In [14]:
def combine_labels(data):
    comb_labels = data['context'].astype(str) + data['datatype'].astype(str) + data['attitude'].astype(str)
    return comb_labels

In [15]:
complete_df['output'] = [get_encoded_labels(X) for X in combine_labels(complete_df)]

In [ ]:
tokenizer= BertTokenizer.from_pretrained("bert-base-cased")

def get_tokenList(df):
    tokenList = []
    for i in range(len(df["q"])):
        q = df.iloc[i]["q"]
        a = df.iloc[i]["a"]
        try:
            tokenList.append(tokenizer(a, padding='max_length', max_length=100, truncation=True, return_tensors="pt", add_special_tokens=True))
        except:
            print(q, "------", a)
            tokenList.append(None)
    return tokenList

In [44]:
complete_df["tokenList"] = get_tokenList(complete_df)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [45]:
complete_df = complete_df.dropna()
complete_df

,topic,idx,q,a,context,datatype,attitude,group,output,tokenList
0,occupation,26876,What if a nursing student doesn't want to be a...,I also have the same idea as you. I am a fresh...,0,0,0,Nursing careers,0,"[input_ids, token_type_ids, attention_mask]"
1,region,210,Are Shanghainese really richer?,"A very serious answer, this question should be...",0,1,2,Shanghainese,6,"[input_ids, token_type_ids, attention_mask]"
2,Gender,7902,How do lesbians make life after 30 not pathetic?,"Invincible and comfortable, I thought about ma...",1,2,2,Homosexual/Lesbian,22,"[input_ids, token_type_ids, attention_mask]"
3,Gender,7658,Where is the way out for homosexuals in China?,I was lucky. The girl has come out with her pa...,1,1,2,Homosexual,18,"[input_ids, token_type_ids, attention_mask]"
4,Gender,8739,Is there any way to discern LES?,"I don't know, I just know that I like my siste...",1,0,0,Lesbian,12,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...,...,...,...,...,...,...
19719,Ethnic culture,20848,How do you view foreigners thinking of Chinese...,Why didn't I understand that? Shiro is not a r...,0,0,0,Complexion,0,"[input_ids, token_type_ids, attention_mask]"
19720,region,1795,What do Northeasterners think of the Northeast...,Hate that iron is not steel. As long as there ...,1,1,3,Northeasterners,19,"[input_ids, token_type_ids, attention_mask]"
19721,Gender,13725,What do you think of the unabashed discriminat...,"There is no need to ask this question, feminis...",0,2,2,Discrimination against men,10,"[input_ids, token_type_ids, attention_mask]"
19722,Gender,10507,Want to know what it's like to have a stay-at-...,"I asked my husband, and he said that I was abl...",0,1,2,housewife,6,"[input_ids, token_type_ids, attention_mask]"


In [ ]:
tokenizer= BertTokenizer.from_pretrained("bert-base-cased")

class Dataset(torch.utils.data.Dataset): 
    def __init__(self,df): 
        #extract our labels from the df 
        # self.contextLabels = df["context"]
        # self.attitudeLabels = df["attitude"]
        # self.dataTypeLabels = df["datatype"]
        self.labels = df["output"]
        #tokenize our texts to the format that BERT expects to get as input 
        #self.texts = df["tokenList"]
        self.texts = df["token_answer"]
        # print(self.texts)

    #def classes(self, get_class):
    def classes(self):
        return self.labels
        # if get_class == 'context':
        #     return self.contextLabels
        # elif get_class == 'attitude':
        #     return self.attitudeLabels
        # elif get_class == "datatype":
        #     return self.dataTypeLabels
    
    def __len__(self):
        return len(self.labels)
        # if get_class == 'context':
        #     return len(self.contextLabels)
        # elif get_class == 'attitude':
        #     return len(self.attitudeLabels)
        # elif get_class == "datatype":
        #     return len(self.dataTypeLabels) 
    
    #fetch a batch of labels
    def get_batch_labels(self,indx):
        return np.array(self.labels[indx])
        # if get_class == 'context':
        #     return np.array(self.contextLabels[indx])
        # elif get_class == 'attitude':
        #     return np.array(self.attitudeLabels[indx])
        # elif get_class == "datatype":
        #     return np.array(self.dataTypeLabels[indx]) 

    # fetch a batch of texts 
    def get_batch_texts(self,indx): 
        return self.texts[indx]

    #get an item with the texts and the label
    def __getitem__(self,indx):

        batch_texts = self.get_batch_texts(indx)
        batch_y = self.get_batch_labels(indx)
        
        return batch_texts, batch_y

In [17]:
torch_tensor_dy = torch.tensor(complete_df['output'].values)


In [18]:
torch_tensor_dy

tensor([ 0,  6, 22,  ..., 10,  6, 11])

In [101]:
def dataset_split(data):
    return np.split(data.sample(frac=1,random_state=42),[int(.8*len(data)), int(.9*len(data))])

In [102]:
df_train, df_valid,df_test = dataset_split(complete_df)

In [24]:
class BertClassifier(torch.nn.Module): 
    def __init__(self,dropout=0.5): 
        super(BertClassifier,self).__init__()
        
        self.bert=BertModel.from_pretrained("bert-base-cased")
        self.dropout = torch.nn.Dropout(dropout)
        # bert output a vector of size 768
        self.lin = torch.nn.Linear(768,24)
        self.relu = torch.nn.ReLU()
        
    def forward(self,input_id,mask): 
        # as output, the bert model give us first the embedding vector of all the tokens of the sequence 
        # second we get the embedding vector of the CLS token.
        # fot a classification task it's enough to use this embedding for our classifier
        _,pooled_output = self.bert(input_ids= input_id,attention_mask = mask,return_dict = False)
        dropout_output = self.dropout(pooled_output)
        linear_output  = self.lin(dropout_output)
        final_layer = self.relu(linear_output)
        
        return final_layer

In [131]:
def train(model, train_data, val_data, learning_rate, epochs=5):
    #creating a custom Dataset objects using the training and validation data
    train, val = Dataset(train_data), Dataset(val_data)
    
    #creating dataloaders
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
            # print(f"the train input : {train_input}")
            # print(f"train label : {train_label}")

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)
            #print(input_id.shape)

            # get the predictions 
            output = model(input_id, mask)

            #the output is a vector of 5 values (categs)
            #print(output)
            #print("the output shape is" ,  output.shape)
            #print(train_label)
            
            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc
            # updating the Gradient Descent and Backpropagation operation
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        # now we evaluate on the validation data
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [132]:
EPOCHS = 5
model = BertClassifier()
learning_rate = 0.001
train(model, df_train, df_valid, learning_rate, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyError: 10179

In [26]:
def evaluate(model,test_df):
    test = Dataset(test_df)
    test_dl = torch.utils.data.DataLoader(test,batch_size=2)
    
    cuda_available = torch.cuda.is_available()
    
    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    if cuda_available:
        model = model.cuda()
    
    total_acc = 0
    for test_input , test_label in tqdm(test_dl):
        test_label = test_label.to(device)
        mask = test_input["attention_mask"].to(device)
        input_id = test_input["input_ids"].squeeze(1).to(device)
        output = model(input_id,mask)
        
        acc = (output.argmax(dim=1) == test_label).sum().item()
        total_acc +=acc 
        
    print(f"Test Accuracy : {total_acc / len(test_df): .3f}")

In [ ]:
evaluate(model,test_df=df_test)

In [90]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 0

# For every sentence...
for sent in X:
  # print(sent)
  input_ids = tokenizer.encode(sent, add_special_tokens=True)
  max_len = max(max_len, len(input_ids))
print('Max sentence length: ', max_len)


Max sentence length:  110


In [91]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
import torch
input_ids = []
attention_masks = []

# For every sentence...
for sent in X:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 115,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                   )
   
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(complete_df['output'].values)

print('Original: ', X[1])
print('Token IDs:', input_ids[1])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  Are Shanghainese really richer?A very serious answer, this question should be why Shanghainese looks richer to others.
Token IDs: tensor([  101,  2024,  8344, 14183,  2428, 26108,  1029,  1037,  2200,  3809,
         3437,  1010,  2023,  3160,  2323,  2022,  2339,  8344, 14183,  3504,
        26108,  2000,  2500,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
 

In [92]:
# Print sentence 0, now as a list of IDs.
print('Original: ', X[0])
print('Token IDs:', input_ids[0])

Original:  What if a nursing student doesn't want to be a nurse?I also have the same idea as you. I am a freshman and haven't practiced yet, but I am slow and timid. I want to be a teacher or take the civil service exam. I am very confused now?
Token IDs: tensor([  101,  2054,  2065,  1037,  8329,  3076,  2987,  1005,  1056,  2215,
         2000,  2022,  1037,  6821,  1029,  1045,  2036,  2031,  1996,  2168,
         2801,  2004,  2017,  1012,  1045,  2572,  1037, 10452,  1998,  4033,
         1005,  1056,  9051,  2664,  1010,  2021,  1045,  2572,  4030,  1998,
         5199,  3593,  1012,  1045,  2215,  2000,  2022,  1037,  3836,  2030,
         2202,  1996,  2942,  2326, 11360,  1012,  1045,  2572,  2200,  5457,
         2085,  1029,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,

In [93]:
input_ids.shape

torch.Size([19722, 115])

In [94]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(input_ids, y, train_size=0.8)
X_train, X_test, y_train, y_test = train_test_split(input_ids, labels, train_size=0.8)

In [95]:
X_train.shape

torch.Size([15777, 115])

In [96]:
y_train

tensor([19, 23, 10,  ..., 10,  6,  0])

In [65]:
X_data = []
for i in range(len(X_train)):
  X_data.append([X_train[i], y_train[i]])

In [66]:
len(y_train)

15777

In [67]:
X_data_test = []
for i in range(len(X_test)):
  X_data_test.append([X_test[i], y_test[i]])

In [68]:
batch_size = 32
train_loader1 = torch.utils.data.DataLoader(X_data, batch_size=batch_size)

In [73]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(115, 768)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(768, 24)


    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [121]:
model = Net()

In [124]:
n_epochs = 25
criterion_fnn = nn.CrossEntropyLoss()
optimizer_fnn = torch.optim.Adam(model.parameters(), lr=0.0001)

In [125]:
import numpy as np
valid_loss_min = np.Inf 

for epoch in range(n_epochs):
    train_loss = 0.0
    model.train() 
    for data, target in train_loader1:
        optimizer_fnn.zero_grad()
        output = model(data.float())
        # loss = criterion_fnn(output, torch.max(target, 1)[1])
        loss = criterion_fnn(output, target.long())
        loss.backward()
        optimizer_fnn.step()
        train_loss += loss.item()*data.size(0)
        
    model.eval()
    # if ((epoch+1)%100 == 0):    
    #     print('Epoch: {} \tTraining Loss: {:.2f}'.format(epoch+1, train_loss/len(train_loader1.dataset)))
    print('Epoch: {} \tTraining Loss: {:.2f}'.format(epoch+1, train_loss/len(train_loader1.dataset)))

Epoch: 1 	Training Loss: 1178.93
Epoch: 2 	Training Loss: 872.40
Epoch: 3 	Training Loss: 758.05
Epoch: 4 	Training Loss: 648.89
Epoch: 5 	Training Loss: 553.50
Epoch: 6 	Training Loss: 472.02
Epoch: 7 	Training Loss: 398.12
Epoch: 8 	Training Loss: 323.44
Epoch: 9 	Training Loss: 262.74
Epoch: 10 	Training Loss: 206.23
Epoch: 11 	Training Loss: 157.95
Epoch: 12 	Training Loss: 120.68
Epoch: 13 	Training Loss: 94.12
Epoch: 14 	Training Loss: 78.18
Epoch: 15 	Training Loss: 71.68
Epoch: 16 	Training Loss: 67.10
Epoch: 17 	Training Loss: 64.81
Epoch: 18 	Training Loss: 64.37
Epoch: 19 	Training Loss: 61.92
Epoch: 20 	Training Loss: 59.91
Epoch: 21 	Training Loss: 57.97
Epoch: 22 	Training Loss: 55.71
Epoch: 23 	Training Loss: 55.01
Epoch: 24 	Training Loss: 52.49
Epoch: 25 	Training Loss: 50.95


In [126]:
test_loader1 = torch.utils.data.DataLoader(X_data_test, batch_size=1)

In [127]:
label_context = []
def q4b_fnn_accuracy(loader, model):
  total_correct = 0
  total_samples = 0
  model.eval()
  cnt = 0
  with torch.no_grad():

    for data, label in loader:
      cnt += 1
      scores = model(data.float())
      _, predictions = scores.max(1)
      label_context.append(predictions.item() )
      pred_arr = (predictions == label)
      for i in pred_arr:
        total_correct += i
    total_samples = len(X_data_test)
    print("Accuracy is" , float(total_correct)/float(total_samples) * 100)
  model.train()
q4b_fnn_accuracy(test_loader1, model)

Accuracy is 20.785804816223067


In [85]:
label_context

[0,
 0,
 0,
 6,
 18,
 0,
 22,
 0,
 0,
 0,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 18,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 23,
 0,
 0,
 0,
 0,
 0,
 0,
 18,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 23,
 0,
 0,
 0,
 0,
 22,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 23,
 12,
 0,
 0,
 18,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 22,
 0,
 0,
 0,
 0,
 17,
 0,
 6,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 18,
 0,
 23,
 0,
 0,
 0,
 0,
 6,
 0,
 6,
 6,
 0,
 0,
 0,
 11,
 0,
 23,
 6,
 6,
 0,
 12,
 0,
 0,
 0,
 0,
 6,
 6,
 6,
 0,
 0,
 6,
 0,
 0,
 6,
 0,
 0,
 6,
 6,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 23,
 0,
 0,
 0,
 23,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 6,
 18,
 0,
 0,
 0,
 6,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 12,
 6,
 0,
 6,
 0,
 0,
 18,
 18,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 23,
 0,
 0,
 23,
 23,
 0,
 6,
 0,
 0,
 6,
 0,
 0,
 0,
 12,
 18,
 0,
 0,
 18,
 6,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 

In [67]:
labels_df = pd.DataFrame({'Labels':label_context})

In [68]:
labels_df

,Labels
0,0
1,22
2,0
3,0
4,0
...,...
155,12
156,0
157,6
158,6


In [69]:
dict_map1 = {v: k for k, v in dict_map.items()}

In [71]:
labels_df1 = labels_df.replace({"Labels": dict_map1})

In [72]:
labels_df1

,Labels
0,000
1,122
2,000
3,000
4,000
...,...
155,100
156,000
157,012
158,012


In [73]:
context = []
datatype = []
attitude = []
for i in range(len(labels_df1)):
  context.append(labels_df1['Labels'][i][0])
  datatype.append(labels_df1['Labels'][i][1])
  attitude.append(labels_df1['Labels'][i][2])

In [74]:
df_final = pd.DataFrame({'context':context, 'datatype':datatype, 'attitude':attitude})

In [75]:
df_final

,context,datatype,attitude
0,0,0,0
1,1,2,2
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
155,1,0,0
156,0,0,0
157,0,1,2
158,0,1,2
